In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
from dask.distributed import Client
import dask.dataframe as dd
from src.data_cleaning import clean_df, lemmatize

In [3]:
RAW_FILE = "sample50k_health_tech.csv"
PROECESSED_PATH = Path("../data/data_processed")
PROCESSED_OUT = PROECESSED_PATH/RAW_FILE.replace('.csv', '')
DF_DASK_PARTITIONS = 24

PROCESSED_OUT.mkdir(exist_ok=True)

In [4]:
client = Client()
client.cluster

In [5]:
df_meta = pd.read_csv(
  Path("../data/subsamples")/RAW_FILE,
  usecols = (0,4),
  nrows=5
  )
df_meta.columns = ["article_id", "article"]
df_meta = df_meta.dropna(subset=['article'])

In [6]:
df_dask = pd.read_csv(
  Path("../data/subsamples")/RAW_FILE,
  usecols = (0,4)
  )
df_dask = dd.from_pandas(df_dask, npartitions=DF_DASK_PARTITIONS)
df_dask.columns = ["article_id", "article"]
df_dask = df_dask.dropna(subset=["article"])
df_dask = df_dask.map_partitions(clean_df, meta=df_meta)
df_dask = df_dask.map_partitions(lemmatize, meta=df_meta)
df_dask.to_json(PROCESSED_OUT/"processed_corpus_chunk_*.json", orient='records', lines=True)

In [7]:
client.close()